In [2]:
import jax
import jax.numpy as jnp
from jax.scipy.optimize import minimize
import jax.numpy as jnp
import jax.scipy as jsp

In [1]:
import subprocess
import sys

#!/home/selfetni/anaconda3/bin/python3.9.19
#print("PYTHON VERSION: ",sys.version)
# Install pyDOE using pip
subprocess.call(['pip', 'install', 'pyDOE'])

#!pip install pyDOE

import tensorflow as tf
import datetime, os
#hide tf logs 
os.environ['TF_CPPclea_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'} 
#0 (default) shows all, 1 to filter out INFO logs, 2 to additionally filter out WARNING logs, and 3 to additionally filter out ERROR logs
import scipy.optimize
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
import time
from pyDOE import lhs         #Latin Hypercube Sampling
import seaborn as sns 
import codecs, json
import math
from numba import jit
# generates same random numbers each time
np.random.seed(1234)
tf.random.set_seed(1234)
import random
import datetime
import shutil
import json
import glob
import re 

print("TensorFlow version: {}".format(tf.__version__))
import random
import scipy.io as sio
from importlib import reload
import PINN  # python files (classes)
import pre_post
from pre_post import *
from PINN import *

2023-08-03 17:10:12.634835: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-03 17:10:12.710467: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-08-03 17:10:12.710483: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-08-03 17:10:13.128096: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

TensorFlow version: 2.11.0


In [2]:
#generate_circles without overlap
@jit(nopython=True)
def generate_circles(mean_r, num_circles, std, Nx, Ny, Nz):
    # Initialize the arrays for the radii and centers of the circles
    R0 = np.zeros(num_circles)
    X_center = np.zeros(num_circles)
    Y_center = np.zeros(num_circles)
    Z_center = np.zeros(num_circles)

    # Generate the first circle randomly
    R0[0] = np.random.normal(loc=mean_r, scale=std)
    X_center[0] = np.random.randint(R0[0], Nx-R0[0])
    Y_center[0] = np.random.randint(R0[0], Ny-R0[0])
    Z_center[0] = np.random.randint(R0[0], Nz-R0[0])

    # Loop through the remaining circles and generate them one at a time
    for i in range(1, num_circles):
        # Flag to indicate whether the new circle overlaps with any existing circles
        overlaps = True
        while overlaps:
            # Generate the radius and center of the new circle randomly
            R0[i] = np.random.normal(loc=mean_r, scale=std)
            X_center[i] = np.random.randint(R0[i], Nx-R0[i])
            Y_center[i] = np.random.randint(R0[i], Ny-R0[i])
            Z_center[i] = np.random.randint(R0[i], Nz-R0[i])

            # Check the new circle against the existing circles
            overlaps = False
            for j in range(i):
                if np.sqrt((X_center[i]-X_center[j])**2 + (Y_center[i]-Y_center[j])**2 ) < (R0[i]+R0[j]): #+ (Z_center[i]-Z_center[j])**2
                    overlaps = True
                    break
    
    return R0, X_center, Y_center, Z_center


In [3]:

if __name__ == '__main__':
    # Create an empty list to store u_pred values
    # Input parameters
    # Grid parameters
    Nx=64
    Ny=64
    Nt=100
    eta=0.075  
    # Define the domain bounds
    lb = np.array([0, 0,0])
    ub = np.array([1, 1,1]) #np.array([Nx, Ny,Nt])
    
    # Phyiscal paramters
    v_n=0.5
    sigma=1
    mu=1e-3
    delta_g=-100
    
    # Numerical parameters 
    N_batches=36 # Total number of data points for 'phi': boundary 
    Nbr_pts_max_per_batch=12500  # maximum number  Collocation points per batch for Scipy optimizer 
    scipy_min_f_pts_per_batch = 500 # minimum number of Collocation points per batch for Scipy optimizer 
    max_ic_scipy_pts=75 #maximum number of IC points per batch for Scipy optimizer 
    N_ini_min_per_batch=10
    
    
    scipy_min_f_pts_per_batch_thresh =0.05  # to delete
    ic_scipy_thresh=0.05 # 
    
    

    num_train_intervals=100
    # Define  Collocations, IC and BC points and Domain bounds
    N_ini =N_batches *num_train_intervals # Total number of data points for 'phi': IC
    N_f = Nbr_pts_max_per_batch * N_batches *num_train_intervals    # 100000 Total number of collocation points : domain
    N_b=500   # Total number of data points for boundary BC
    N_ini_min_per_batch=3
    
    
        
   # Total number of data points for 'phi': boundary BC

    # get radius and coordinates
    R0, X_center, Y_center,Z_center =\
          generate_circles(mean_r=0.3,num_circles=1, std=0, Nx=Nx, Ny=Ny,Nz=100)
    X_center, Y_center=[0.5],[0.5] # single grain

    x = np.linspace(lb[0], ub[0], Nx),
    y = np.unique(np.linspace(lb[1], ub[1], Ny))
    t = np.unique(np.linspace(lb[2], ub[2], Nt))
    x=np.expand_dims(x, axis=1)
    y=np.expand_dims(y, axis=1)
    t=np.expand_dims(t, axis=1)
    X, Y,T= np.meshgrid(x,y,t)

    tb = np.linspace(start=lb[2], stop=ub[2], num=N_b, endpoint=True)
    tb = np.expand_dims(tb, axis=1)
    
    # set the saving paths and erase older results
    global pathOutput
    pathOutput = os.path.join(os.getcwd(),'save_figs')
    if not os.path.isdir(pathOutput):
        os.mkdir(pathOutput)
    # to store the weights for each time interval 
    path_weights= os.path.join(os.getcwd(),'weights')
    if not os.path.isdir(path_weights):
        os.mkdir(path_weights)

    # load PrePost class
    reload(pre_post) # for re-execution after modif
    from pre_post import *
    Pre_Post=PrePost( X=X,T=None, lb=lb, ub=ub, Nx=Nx,Ny=Ny,x=x,y=y, eta=eta,\
        phi_true=None,R0=R0)

    # set the save paths and erase older results
    #Pre_Post.EraseFile(path=pathOutput)
    #Pre_Post.EraseFile(path=path_weights)
 
    # get phi_0
    phi_0, X_ini_all=Pre_Post.init_micro_cir(X_center,Y_center, Z_center,N_ini,Nx,Ny,x,y,lb,ub) 
    
    # plot the true solution
    #Pre_Post.plot_exact(path=pathOutput)

    # plot the initial micro
    Pre_Post.plot_init(X_ini_all,phi_0,Nx,Ny,path=pathOutput)
    
    # get the training data
    X_f_train, X_ini_train,X_lb_train,X_ub_train,X_rtb_train,X_ltb_train,\
        phi_ini_train, X_ini_train_all, phi_ini_train_all=Pre_Post.set_training_data(x,y,X_ini_all,N_ini,phi_0,N_f,tb,lb,ub)
 
    # Plot Collocation_IC_BC points
    #Pre_Post.plot_Collocation_IC_BC(Nx,Ny,x,y,X_ini_train,X_f_train,X_lb_train,X_ub_train,\
    #                                X_rtb_train,X_ltb_train,phi_0,phi_ini_train,path=pathOutput)
      
    # Testing spatio-temporal domain
    X_phi_test = np.hstack((X.flatten()[:,None],Y.flatten()[:,None], T.flatten()[:,None])) 
    #tf.print(X_phi_test)
 
 
    



X_f_train: (45000000, 3), X_ini_train: (3600, 3), X_lb_train: (500, 3), X_ub_train: (500, 3), phi_ini_train: (3600, 1)


2023-06-19 11:56:23.286529: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-06-19 11:56:23.286578: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-06-19 11:56:23.286608: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-06-19 11:56:23.286636: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2023-06-19 11:56:23.286662: W tensorfl

In [4]:
# load PINN class
import PINN
reload(PINN)  # mandatory to reload content at each re-call atfer modification
from PINN import *

# Build PINN 
# Build PINN 
layers = np.array([3,128,128, 128,128,128,128,1])  # Network
PINN_ = Sequentialmodel(layers=layers, X_f_train=X_f_train, X_ini_train=X_ini_train,\
                        X_lb_train=X_lb_train, X_ub_train=X_ub_train,\
                        X_ltb_train=X_ltb_train, X_rtb_train=X_rtb_train,\
                        phi_0=phi_0,phi_ini_train=phi_ini_train, N_ini=N_ini,X_u_test=X_phi_test,\
                        X_ini_train_all=X_ini_train_all, phi_ini_train_all=phi_ini_train_all,\
                        X=X,T=T,x=x,y=y,lb=lb, ub=ub, mu=mu, sigma=sigma, delta_g=delta_g,\
                                    R0=R0,X_center=X_center,Y_center=Y_center,eta=eta,Nx=Nx,Ny=Ny,Nt=Nt,phi_sol=None)

In [6]:
PINN_.save_predictions(1,pathOutput,PINN_.X_u_test, PINN_.X_ini,PINN_.phi_ini,N_b,0,1)

In [3]:
N_batches= 16 
limits = []
num_x_intervals = int(np.ceil(np.sqrt(N_batches)))
num_y_intervals = int(np.ceil(np.sqrt(N_batches)))

step_x = 1 / num_x_intervals
step_y = 1/ num_y_intervals
for i in range(int(np.sqrt(N_batches))):
    for j in range(int(np.sqrt(N_batches))):
        #tf.print("index", index)
        x_min = i * step_x
        x_max = (i + 1) * step_x
        y_min = j * step_y
        y_max = (j + 1) * step_y
        limits.append([x_min, x_max, y_min, y_max])

check  4 4


In [4]:
limits

[[0.0, 0.25, 0.0, 0.25],
 [0.0, 0.25, 0.25, 0.5],
 [0.0, 0.25, 0.5, 0.75],
 [0.0, 0.25, 0.75, 1.0],
 [0.25, 0.5, 0.0, 0.25],
 [0.25, 0.5, 0.25, 0.5],
 [0.25, 0.5, 0.5, 0.75],
 [0.25, 0.5, 0.75, 1.0],
 [0.5, 0.75, 0.0, 0.25],
 [0.5, 0.75, 0.25, 0.5],
 [0.5, 0.75, 0.5, 0.75],
 [0.5, 0.75, 0.75, 1.0],
 [0.75, 1.0, 0.0, 0.25],
 [0.75, 1.0, 0.25, 0.5],
 [0.75, 1.0, 0.5, 0.75],
 [0.75, 1.0, 0.75, 1.0]]

#### FD resolution